In [1]:
# tryout script to do the whole pipeline on the prf data of one sub
# after try to generalize for all tasks 
# put in Nipype format (do mapNodes because then I can iterate throught files, nicer)

In [2]:
# import packages
from nilearn import image, plotting
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
import numpy as np
import nibabel as nib
from spynoza.filtering.nodes import Savgol_filter#, Savgol_filter_confounds
from spynoza.conversion.nodes import Percent_signal_change
from nipype import Node, Function
import nipype.pipeline as pe
import nipype.interfaces.io as nio
from bids.grabbids import BIDSLayout

#from nipype.interfaces import fsl
#from nipype.interfaces import freesurfer
#from nipype.interfaces.utility import Function, IdentityInterface


In [3]:
# save relevant params used in json file, 
# afterwards just need to call the field of the list/dict
import json
analysis_params = {}
json_info = open('SBref_analysis_params.json','r').read()
analysis_params.update(json.loads(json_info))

In [4]:
#analysis_params

In [5]:
# define functions
def SGfilt_confound(confounds, tr, polyorder=3, deriv=0, window_length=120):
    import pandas as pd
    from scipy.signal import savgol_filter
    import numpy as np
    import os

    confounds_table = pd.read_csv(confounds, sep='\t', na_values='n/a') #added this line to original spynoza func because it was giving error when str transformed to float
    
    window = np.int(window_length / tr)

    # Window must be odd
    if window % 2 == 0:
        window += 1

    confounds_filt = savgol_filter(confounds_table, window_length=window, polyorder=polyorder,
                              deriv=deriv, axis=0, mode='nearest')

    new_name = os.path.basename(confounds).split('.')[:-1][0] + '_sg.tsv'
    out_file = os.path.abspath(new_name)

    confounds_table = np.asarray(confounds_table).astype(np.float64) # needed to convert to float to support next operation
    pd.DataFrame(confounds_table - confounds_filt).to_csv(out_file, sep='\t', index=False)

    return out_file

In [6]:
# define paths and variables
sub_list = ['2']
ses_list = ['1']
task='prf'

source_pth = '/home/neuro/projects/data/derivatives/fmriprep/'
dest_pth = '/home/neuro/projects/data/derivatives/post_fmriprep/'
sub_num = str(sub_list[0]).zfill(2)
ses_num = str(ses_list[0]).zfill(2)

In [7]:
#gets preprocessed niftis from derivative folder
layout = BIDSLayout(source_pth)
funcdata = layout.get(subject=sub_num,task=task,extensions='space-T1w_preproc.nii.gz',return_type='file') # list of paths to functionals for this sub
confdata = layout.get(subject=sub_num,task=task,extensions='.tsv',return_type='file')  # list of paths to confounds for this sub

In [8]:
funcdata

['/home/neuro/projects/data/derivatives/fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-01_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-02_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-03_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-04_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-05_bold_space-T1w_preproc.nii.gz']

In [9]:
# do some HP filtering (Savitzky–Golay) on bold + confounds
# define Nodes
SG_filter = pe.MapNode(interface=Savgol_filter,
                       name='savgol_filt',
                       iterfield=['in_file'])
SG_filter_confounds = pe.MapNode(Function(function=SGfilt_confound),
                       name='sgfilt_confound',
                       iterfield=['confounds']) #iterfied needs different name because of temp dir

In [10]:
# define settings, right ones? 
SG_filter.inputs.polyorder = SG_filter_confounds.inputs.polyorder = analysis_params['sgfilter_polyorder']
SG_filter.inputs.deriv = SG_filter_confounds.inputs.deriv = analysis_params['sgfilter_deriv']
SG_filter.inputs.window_length = SG_filter_confounds.inputs.window_length = analysis_params['sgfilter_window_length']

if (sub_num=='01' or sub_num=='03') and ses_num=='01': # exception for some initial subjects' sessions
    SG_filter.inputs.tr = SG_filter_confounds.inputs.tr = 1.5
else:
    SG_filter.inputs.tr = SG_filter_confounds.inputs.tr = analysis_params['TR'] 

SG_filter.inputs.in_file = funcdata
SG_filter_confounds.inputs.confounds = confdata


In [11]:
# run and print outputs
res_SG_filter = SG_filter.run()
#res_SG_filter.outputs.out_file

190117-12:20:43,751 nipype.workflow INFO:
	 [Node] Setting-up "savgol_filt" in "/tmp/tmp77pv0_le/savgol_filt".
190117-12:20:43,766 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt0" in "/tmp/tmp77pv0_le/savgol_filt/mapflow/_savgol_filt0".
190117-12:20:43,774 nipype.workflow INFO:
	 [Node] Running "_savgol_filt0" ("nipype.interfaces.utility.wrappers.Function")
190117-12:20:57,744 nipype.workflow INFO:
	 [Node] Finished "_savgol_filt0".
190117-12:20:57,753 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt1" in "/tmp/tmp77pv0_le/savgol_filt/mapflow/_savgol_filt1".
190117-12:20:57,758 nipype.workflow INFO:
	 [Node] Running "_savgol_filt1" ("nipype.interfaces.utility.wrappers.Function")
190117-12:21:11,87 nipype.workflow INFO:
	 [Node] Finished "_savgol_filt1".
190117-12:21:11,92 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt2" in "/tmp/tmp77pv0_le/savgol_filt/mapflow/_savgol_filt2".
190117-12:21:11,97 nipype.workflow INFO:
	 [Node] Running "_savgol_filt2" ("nipype.int

In [12]:
res_SG_filter_confounds = SG_filter_confounds.run()
#res_SG_filter_confounds.outputs.out

190117-12:22:19,226 nipype.workflow INFO:
	 [Node] Setting-up "sgfilt_confound" in "/tmp/tmpc7fvcwcc/sgfilt_confound".
190117-12:22:19,241 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound0" in "/tmp/tmpc7fvcwcc/sgfilt_confound/mapflow/_sgfilt_confound0".
190117-12:22:19,249 nipype.workflow INFO:
	 [Node] Running "_sgfilt_confound0" ("nipype.interfaces.utility.wrappers.Function")
190117-12:22:19,304 nipype.workflow INFO:
	 [Node] Finished "_sgfilt_confound0".
190117-12:22:19,309 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound1" in "/tmp/tmpc7fvcwcc/sgfilt_confound/mapflow/_sgfilt_confound1".
190117-12:22:19,317 nipype.workflow INFO:
	 [Node] Running "_sgfilt_confound1" ("nipype.interfaces.utility.wrappers.Function")
190117-12:22:19,348 nipype.workflow INFO:
	 [Node] Finished "_sgfilt_confound1".
190117-12:22:19,353 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound2" in "/tmp/tmpc7fvcwcc/sgfilt_confound/mapflow/_sgfilt_confound2".
190117-12:22:19,362 nipy

In [13]:
#Convert data to percent signal change
#define Nodes
psc = pe.MapNode(interface=Percent_signal_change, 
              name='percent_signal_change',
             iterfield = ['in_file'])

In [14]:
# define settings
psc.inputs.func = 'median'
psc.inputs.in_file = res_SG_filter.outputs.out_file


In [15]:
# run and print outputs
res_psc = psc.run()
#res_psc.outputs.out_file

190117-12:22:27,707 nipype.workflow INFO:
	 [Node] Setting-up "percent_signal_change" in "/tmp/tmpyc39sd8b/percent_signal_change".
190117-12:22:27,720 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change0" in "/tmp/tmpyc39sd8b/percent_signal_change/mapflow/_percent_signal_change0".
190117-12:22:27,727 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change0" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190117-12:22:38,869 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change0".
190117-12:22:38,872 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change1" in "/tmp/tmpyc39sd8b/percent_signal_change/mapflow/_percent_signal_change1".
190117-12:22:38,877 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change1" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190117-12:22:49,768 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change1".
190117-12:22:49,772 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change2" in "/tmp/tmpyc39sd8b/percent_signal_change/mapflow/_percent_signal_change2".
190117-12:22:49,777 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change2" ("nipype.interfaces.utility.wrappers.Function")


...

190117-12:23:00,893 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change2".
190117-12:23:00,896 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change3" in "/tmp/tmpyc39sd8b/percent_signal_change/mapflow/_percent_signal_change3".
190117-12:23:00,900 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change3" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190117-12:23:12,463 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change3".
190117-12:23:12,468 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change4" in "/tmp/tmpyc39sd8b/percent_signal_change/mapflow/_percent_signal_change4".
190117-12:23:12,471 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change4" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190117-12:23:24,24 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change4".
190117-12:23:24,29 nipype.workflow INFO:
	 [Node] Finished "percent_signal_change".


In [37]:
def nistats_confound_pca_glm(nifti_file, confounds_file,output_pth):# which_confounds,output_pth):    # function adapted from utils.py in pearl_7T Git
    
    import pandas as pd
    import nibabel as nb
    import numpy as np
    import os
    from nistats.regression import OLSModel
    from scipy.stats import zscore
    from nilearn.image import math_img
    from nilearn.plotting import plot_stat_map
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA

    infile = nb.load(nifti_file)
    mean_img = math_img('np.mean(infile, axis=-1)', infile=infile)
    in_data = infile.get_data().astype(np.float32)
    
    confounds_table = pd.read_csv(confounds_file, sep='\t', na_values='n/a')#[which_confounds] #select subgroup of confounds
    
    # Z-SCORING 1ST, THEN OBTAINING COMPONENTS
    # we assume the confounds nor the nifti_file need temporal filtering 
    z_conf = confounds_table.apply(zscore) #do zscore of each value in the sample (scale data matrix)
    
    pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that at least 95% of the variance is retained.
    pca_confs = pca.fit_transform(np.nan_to_num(z_conf))
    num_comp = pca.n_components_

    all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
    all_confs['intercept'] = np.ones(infile.shape[-1]) #add an intercept column filled with ones
    
    om = OLSModel(np.array(all_confs)) #create a least squares model based on nuisances
    om_rr = om.fit(in_data.reshape((-1,infile.shape[-1])).T) #fit bold data to model
    
    resid_img = nb.Nifti1Image(om_rr.resid.T.reshape(infile.shape).astype(np.float32), affine=infile.affine, header=infile.header)
    cleaned_img = math_img('(resid_img + mean_img[...,np.newaxis]).astype(np.float32)', resid_img=resid_img, mean_img=mean_img)
    
    output_nifti = output_pth+os.path.basename(nifti_file).replace('.nii.gz', '_nuis.nii.gz')
    cleaned_img.to_filename(output_nifti)
    
    output_pdf = output_pth+os.path.basename(confounds_file).replace('.tsv', '_sd-diff.pdf')
    f = plt.figure(figsize=(24,6))
    plot_stat_map(math_img('(infile.std(axis=-1)-cleaned_img.std(axis=-1))/infile.std(axis=-1)', infile=infile, cleaned_img=cleaned_img), 
                bg_img=mean_img, figure=f, cut_coords=(0,0,0), threshold=0.125, vmax=1, cmap='viridis', output_file=output_pdf)
    
    return output_pdf, output_nifti
    

In [166]:

# # STANDARD SCALING 1ST, THEN OBTAINING COMPONENTS
# # we assume the confounds nor the nifti_file need temporal filtering 
# from sklearn.preprocessing import StandardScaler
# stand_conf = StandardScaler().fit_transform(np.nan_to_num(confounds_table))

# from sklearn.decomposition import PCA
# pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that 95% of the variance is retained.
# pca_confs = pca.fit_transform(np.nan_to_num(stand_conf))
# num_comp = pca.n_components_

# print(pca.explained_variance_)
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_ratio_.cumsum())

# all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
# print(all_confs)

In [165]:

# # NO SCALING, JUST OBTAINING COMPONENTS
# # we assume the confounds nor the nifti_file need temporal filtering 

# from sklearn.decomposition import PCA
# pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that 95% of the variance is retained.
# pca_confs = pca.fit_transform(np.nan_to_num(confounds_table))
# num_comp = pca.n_components_

# print(pca.explained_variance_)
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_ratio_.cumsum())

# all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
# print(all_confs)

In [38]:
# Use confounds as nuisance regressors in a GLM
#define Node
confGLM = pe.MapNode(Function(input_names=['nifti_file', 'confounds_file', 'output_pth'],#'which_confounds','output_pth'], 
                              output_names=['output_pdf', 'output_nifti'],
                             function=nistats_confound_pca_glm),
                             name='nistats_confound_pca_glm', 
                             iterfield=["nifti_file", "confounds_file"])

In [39]:
# define settings
#confGLM.inputs.which_confounds = analysis_params['nuisance_columns']
confGLM.inputs.nifti_file = res_psc.outputs.out_file
confGLM.inputs.confounds_file = res_SG_filter_confounds.outputs.out
confGLM.inputs.output_pth = dest_pth+'sub-'+sub_num+'/ses-'+ses_num+'/func/'

In [51]:
# run and print outputs
res_confGLM = confGLM.run()

190117-15:08:19,267 nipype.workflow INFO:
	 [Node] Setting-up "nistats_confound_pca_glm" in "/tmp/tmpto_hg45v/nistats_confound_pca_glm".
190117-15:08:19,273 nipype.workflow INFO:
	 [Node] "nistats_confound_pca_glm" found cached.


In [54]:
res_confGLM.outputs.output_nifti

['/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-01_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-02_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-03_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-04_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-prf_run-05_bold_space-T1w_preproc_sg_psc_nuis.nii.gz']